In [7]:
from sklearn.datasets import load_iris
iris = load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [8]:
import pandas as pd

df = pd.DataFrame(iris["data"], columns=iris["feature_names"])
df["target"] = iris["target"]
df = df.dropna()
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(["target"], axis=1), df["target"], test_size=0.3, random_state=12345)

print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (105, 4)
y_train shape: (105,)
X_test shape: (45, 4)
y_test shape: (45,)


In [10]:
import numpy as np

def euclidean_dist(x1, x2):
    """
    Args:
        x1: shape (number of data in x1, number of features)
        x2: shape (number of data in x2, number of features)
    Return:
        dist: shape (number of data in x1, number of data in x2)
    """
    x1_expand = x1[:, np.newaxis, :]
    x2_expand = x2[np.newaxis, :, :]
    difference = x1_expand - x2_expand
    sum_of_squared_diff = np.sum(difference**2, axis=2)
    dist = np.sqrt(sum_of_squared_diff)
    return dist

In [11]:
from collections import Counter

class KNNClassifier():
  def __init__(self, k, knn_type, attenuation=1):
    self.k = k
    self.knn_type = knn_type
    self.num_classes = None
    self.classes = None
    self.membership = None
    self.X = None
    self.y = None
    self.attenuation = attenuation
  
  def fit(self, X, y):
    self.X = X
    self.y = y
    self.classes = np.sort(np.unique(y))
    self.num_classes = len(self.classes)
    num_data, num_feat = X.shape
    
    if self.knn_type == "fuzzy":
      self.membership = np.zeros((num_data, self.num_classes))
      dist = euclidean_dist(X, X)
      for i in range(num_data):
        # 找到第 i 個樣本 X_i 的 k 個最近鄰居的索引
        # 排序距離並取出前 k+1 個索引 (包含自己)
        # np.argsort 會返回排序後的索引
        sorted_indices = np.argsort(dist[i, :])
        # 排除 i 樣本自身 (距離為 0)，取得 k 個最近鄰居的索引
        # 假設 i 總是在 sorted_indices[0] (距離為 0)
        neighbor_indices = sorted_indices[1:self.k+1]
        neighbor_labels = y[neighbor_indices]
        # 統計 k 個鄰居中各類別的數量 (k_j)
        label_counts = Counter(neighbor_labels)
        current_class_label = y[i]
        for class_label in self.classes:
            k_j = label_counts.get(class_label, 0)
            if class_label == current_class_label:
              self.membership[i, class_label] = 0.51 + (k_j/self.k)*0.49
            else:
              self.membership[i, class_label] = (k_j/self.k)*0.49
      
  def predict(self, X_test):
    dist = euclidean_dist(X_test, self.X)
    predictions = []
    
    for i in range(X_test.shape[0]):
      # get the indices and labels of k neighbors
      sorted_indices = np.argsort(dist[i, :])
      neighbor_indices = sorted_indices[:self.k]
      neighbor_labels = self.y[neighbor_indices]
      
      prob = np.zeros(self.num_classes)
      
      if self.knn_type == "voting":
        label_counts = Counter(neighbor_labels)
        for class_label in self.classes:
          label_cnt = label_counts.get(class_label, 0)
          prob[class_label] = label_cnt / self.k
          
      elif self.knn_type == "distance_weighted":
        neighbor_dist = dist[i, neighbor_indices]
        dist_weights = 1 / (neighbor_dist + 1e-8)**self.attenuation
        for j, class_label in enumerate(self.classes):
          prob[class_label] += dist_weights[j]
        prob = prob / np.sum(dist_weights)
        
      elif self.knn_type == "fuzzy":
        neighbor_dist = dist[i, neighbor_indices]
        dist_weights = 1 / (neighbor_dist + 1e-8)**self.attenuation
        neighbor_membership = self.membership[neighbor_indices, :] # (k, num_class)
        
        for class_label in self.classes:
          for j in range(self.k):
            prob[class_label] += neighbor_membership[j][class_label] * dist_weights[j]
        prob = prob / np.sum(dist_weights)
      
      pred_class = np.argmax(prob)
      predictions.append(pred_class)
    
    predictions = np.array(predictions)
    return predictions


In [12]:
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

model = KNNClassifier(k=5, knn_type="fuzzy")
model.fit(X=X_train_np, y=y_train_np)
predictions = model.predict(X_test=X_test_np)
acc = np.sum(predictions==y_test_np) / y_test_np.shape[0]
print(acc)

0.9777777777777777
